# Processing Big Data - Data Ingestion
© Explore Data Science Academy

## Honour Code
I {**Palesa**, **Nzimande**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.



## Context 

To work constructively with any dataset, one needs to create an ingestion profile to make sure that the data at the source can be readily consumed. For this section of the predict, as the Data Engineer in the team, you will be required to design and implement the ingestion process. For the purposes of the project the AWS cloud storage service, namely, the S3 bucket service will act as your data source. All the data required can be found [here](https://processing-big-data-predict-stocks-data.s3.eu-west-1.amazonaws.com/stocks.zip).

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/DataIngestion.jpg"
     alt="Data Ingestion"
     style="float: center; padding-bottom=0.5em"
     width=40%/>
     <p><em>Figure 1. Data Ingestion</em></p>
</div>

Your manager, Gnissecorp Atadgib, knowing very well that you've recently completed your Data Engineering qualification, asks you to make use of Apache Spark for the ingestion as well as the rest of the project. His rationale being, that stock market data is generated every day and is quite time-sensitive and would require scalability when deploying to a production environment. 

## Dataset - US Nasdaq




<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/Nasdaq.png"
     alt="Nasdaq"
     style="float: center; padding-bottom=0.5em"
     width=50%/>
     <p><em>Figure 2. Nasdaq</em></p>
</div>

The data that you will be working with is a historical snapshot of market data taken from the Nasdaq electronic market. This dataset contains historical daily prices for all tickers currently trading on Nasdaq. The up-to-date list can be found on their [website](https://www.nasdaq.com/)


The provided data contains price data dating back from 02 January 1962 up until 01 April 2020. The data found in the S3 bucket has been stored in the following structure:

```
     stocks/<Year>/<Month>/<Day>/stocks.csv
```
Each CSV file for every trading day contains the following details:
- **Date** - specifies trading date
- **Open** - opening price
- **High** - maximum price during the day
- **Low** - minimum price during the day
- **Close** - close price adjusted for splits
- **Adj Close** - close price adjusted for both dividends and splits
- **Volume** - the number of shares that changed hands during a given day

## Basic initialisation
To get you started, let's import some basic Python libraries as well as Spark modules and functions.

In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

Remember that we need a `SparkContext` and `SparkSession` to interface with Spark.
We will mostly be using the `SparkContext` to interact with RDDs and the `SparkSession` to interface with Python objects.

> ℹ️ **Instructions** ℹ️
>
>Initialise a new **Spark Context** and **Session** that you will use to interface with Spark.

In [3]:
# Create a SparkSession object
spark = SparkSession.builder \
    .appName("StockMarketDataIngestion") \
    .getOrCreate()

# Get a SparkContext from the SparkSession
sc = spark.sparkContext



## Investigate dataset schema
At this point, it is enough to read in a single file to ascertain the data structure. You will be required to use the information obtained from the small subset to create a data schema. This data schema will be used when reading the entire dataset using Spark.

> ℹ️ **Instructions** ℹ️
>
>Make use of Pandas to read in a single file and investigate the plausible data types to be used when creating a Spark data schema. 
>
>*You may use as many coding cells as necessary.*

In [5]:
# Assuming you have downloaded the data from the S3 bucket locally
# Replace "path/to/file.csv" with the actual file path you downloaded

import pandas as pd

# Read the first CSV file
df = pd.read_csv("C:\Users\pales\OneDrive\Documents\School\Explore AI\PBD\stocks")

# Print the first few rows to get a glimpse of the data
print(df.head())

# Investigate data types of each column
print(df.dtypes)




FileNotFoundError: [Errno 2] No such file or directory: 'stocks/<Year>/<Month>/<Day>/stocks.csv'

## Read CSV files

When working with big data, it is often not tenable to keep processing an entire data batch when you are in the process of development - this can be quite time-consuming. If the data is uniform, it is sufficient to work with a smaller subset to create basic functionality. Your manager has identified the year **1962** to perform the initial testing for data ingestion. 

> ℹ️ **Instructions** ℹ️
>
>Read in the data for **1962** using a data schema that purely uses string data types. You will be required to convert to the appropriate data types at a later stage.
>
>*You may use as many coding cells as necessary.*

In [7]:
# Define the base path for the S3 bucket data
base_path = "(https://processing-big-data-predict-stocks-data.s3.eu-west-1.amazonaws.com/stocks.zip"

# Define a schema with string data types
schema = StructType([
    StructField("Date", StringType(), True),
    StructField("Open", StringType(), True),
    StructField("High", StringType(), True),
    StructField("Low", StringType(), True),
    StructField("Close", StringType(), True),
    StructField("Adj Close", StringType(), True),
    StructField("Volume", StringType(), True)
])

# Read the data for 1962 from the S3 bucket using Spark
df = spark.read.csv(base_path, schema=schema, header=True)

df.show()


IllegalArgumentException: java.net.URISyntaxException: Illegal character in scheme name at index 0: (https://processing-big-data-predict-stocks-data.s3.eu-west-1.amazonaws.com/stocks.zip

## Update column names
To make the data easier to work with, you will need to make a few changes:
1. Column headers should all be in lowercase; and
2. Whitespaces should be replaced with underscores.


> ℹ️ **Instructions** ℹ️
>
>Make sure that the column headers are all in lowercase and that any whitespaces are replaced with underscores.
>
>*You may use as many coding cells as necessary.*

In [9]:
# Get the current column names
current_columns = df.columns

# Create a dictionary to map old column names to new column names
column_name_mapping = {old_column: old_column.lower().replace(" ", "_") for old_column in current_columns}

# Rename the columns using a loop
for old_column, new_column in column_name_mapping.items():
    df = df.withColumnRenamed(old_column, new_column)

# Show the updated DataFrame with only the schema (omitting data rows)
df.printSchema()

# You can optionally display the DataFrame content as well:
# df.show()

NameError: name 'df' is not defined

## Null Values
Null values often represent missing pieces of data. It is always good to know where your null values lie - so you can quickly identify and remedy any issues stemming from these.

> ℹ️ **Instructions** ℹ️
>
>Write code to count the number of null values found in each column.
>
>*You may use as many coding cells as necessary.*

In [6]:
# Count null values in each column using Spark SQL functions
null_counts = df.select(F.col("*")).isna().sum()

# Display the null counts for each column
null_counts.show()


## Data type conversion - The final data schema

Now that we have identified the number of missing values in the data set, we'll move on to convert our data schema to the required data types. 

> ℹ️ **Instructions** ℹ️
>
>Use typecasting to convert the string data types in your current data schema to more appropriate data types.
>
>*You may use as many coding cells as necessary.*

In [7]:
from pyspark.sql.functions import col

# Define the final data schema with appropriate data types
final_schema = StructType([
    StructField("column1", IntegerType(), True),  # Example: Convert to IntegerType
    StructField("column2", DoubleType(), True),   # Example: Convert to DoubleType
    # Add more StructField definitions for each column in your dataset with appropriate data types
])

# Apply type casting to each column
for field in final_schema.fields:
    column_name = field.name
    column_type = field.dataType
    df = df.withColumn(column_name, col(column_name).cast(column_type))

# Verify the updated data schema
df.printSchema()


## Consolidate missing values
We have to check if the data type conversion above was done correctly.
If the casting was not successful, a null value gets inserted into the dataframe. You can thus check for successful conversion by determining if any null values are included in the resulting dataframe.


> ℹ️ **Instructions** ℹ️
>
>Write code to compare the number of invalid entries (nulls) pre-conversion and post-conversion.
>
>*You may use as many coding cells as necessary.*

In [8]:
from pyspark.sql.functions import col

# Define the final data schema with appropriate data types
final_schema = StructType([
    StructField("column1", IntegerType(), True),  # Example: Convert to IntegerType
    StructField("column2", DoubleType(), True),   # Example: Convert to DoubleType
    # Add more StructField definitions for each column in your dataset with appropriate data types
])

# Apply type casting to each column
for field in final_schema.fields:
    column_name = field.name
    column_type = field.dataType
    df = df.withColumn(column_name, col(column_name).cast(column_type))

# Verify the updated data schema
df.printSchema()


Here you should be able to see if any of your casts went wrong. 
Do not attempt to correct any missing values at this point. This will be dealt with in later sections of the predict.

## Generate parquet files
When writing in Spark, we typically use parquet format. This format allows parallel writing using Spark's optimisation while maintaining other useful things like metadata.

When writing, it is good to make sure that the data is sufficiently partitioned. 

Generally, data should be partitioned with one partition for every 200MB of data, but this also depends on the size of your cluster and executors. 


### Check the size of the dataframe before partitioning

In [9]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer

In [10]:
rdd = df.rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
obj = rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)
size = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(obj)
size_MB = size/1000000
partitions = max(int(size_MB/200), 2)
print(f'The dataframe is {size_MB} MB')

NameError: name 'df' is not defined

### Write parquet files to the local directory
> ℹ️ **Instructions** ℹ️
>
> Use the **coalesce** function and the number of **partitions** derived above to write parquet files to your local directory 
>
>*You may use as many coding cells as necessary.*

In [10]:
# Specify the local directory where you want to save the Parquet files
output_directory = "path_to_local_directory"  # Replace with your desired local directory path

# Specify the number of partitions
num_partitions = 4  # Replace with the number of partitions you derived above

# Use the coalesce function to reduce the number of partitions
df_coalesced = df.coalesce(num_partitions)

# Write Parquet files to the local directory
df_coalesced.write.parquet(output_directory, mode="overwrite")

# Print a message indicating successful completion
print(f"Parquet files written to {output_directory}")


NameError: name 'df' is not defined